# Customize Graph Convolution using Message Passing APIs
# Message Passing API로 그래프 컨볼루션 커스터마이징하기


이전 세션까지, built-in된 [graph convolution modules](https://docs.dgl.ai/api/python/nn.pytorch.html#module-dgl.nn.pytorch.conv)을 사용해 다중 레이어 그래프 뉴럴넷을 구축했습니다.  
하지만, 때때로 이웃 정보를 통합하는 새로운 방법을 개발하고 싶을 수도 있겠죠.  
DGL의 message passing API는 이런 상황을 위해 설계되었습니다.  

이 튜토리얼에서, 이런 것들을 배울 수 있습니다.  

* DGL의 `nn.SAGEConv` 모듈의 내부는 어떻게 돌아갈까?  
* DGL의 message passing API
* 새로운 그래프 컨볼루션 모듈 설계하기

In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

Using backend: pytorch


## Message passing과 GNN

DGL은 [Gilmer et al.](https://arxiv.org/abs/1704.01212)에 의해 제안된 Message Passing Neural Network에서 고안된  *message passing 패러다임*을 따릅니다.  
기본적으로, 연구진은 많은 GNN 모델이 다음 프레임워크에 들어맞는다는 것을 발견했습니다.  

$$
m_{u\sim v}^{(l)} = M^{(l)}\left(h_v^{(l-1)}, h_u^{(l-1)}, e_{u\sim v}^{(l-1)}\right)
$$

$$
m_{v}^{(l)} = \sum_{u\in\mathcal{N}(v)}m_{u\sim v}^{(l)}
$$

$$
h_v^{(l)} = U^{(l)}\left(h_v^{(l-1)}, m_v^{(l)}\right)
$$

DGL은 $M^{(l)}$ 을 *message function*라 부르며, $\sum$을 the *reduce function*이라 부릅니다.   

여기서 $\sum$은 어떤 함수든 표현할 수 있으며 꼭 반드시 summation일 필요는 없습니다.

가령, GraphSAGE 모델은 다음의 수식적인 형태를 갖고 있습니다.

$$
h_{\mathcal{N}(v)}^k\leftarrow \text{Average}\{h_u^{k-1},\forall u\in\mathcal{N}(v)\}
$$

$$
h_v^k\leftarrow \text{ReLU}\left(W^k\cdot \text{CONCAT}(h_v^{k-1}, h_{\mathcal{N}(v)}^k) \right)
$$

message passing이 유방향적이라는 것을 볼 수 있죠.  
즉, 한 노드 $u$에서 $v$로 보내진 메시지는 반대 방향인 노드 $v$에서 노드 $u$로 보내진 다른 메시지와 꼭 같을 필요는 없다는 말입니다.  

DGL 그래프는 message passing을 수행하는 데 사용할 `srcdata` 와 `dstdata`라는 녀석을 제공합니다.  
먼저 인풋 노드 피처를 `srcdata`에 넣고, message passing을 수행하면,  
`dstdata`로부터 message passing의 결과를 가져올 수 있습니다.

<div class="alert alert-info">
    <b>주의: </b> 전체 그래프(full graph)의 message passing에서, 인풋 노드와 아웃풋 노드는 전체 노드 집합입니다. 그러므로, 동질적(homogeneous) 그래프(즉 오직 1개의 노드 타입과 1개의 엣지 타입만을 가지고 있는 그래프)의 <code>srcdata</code> 와 <code>dstdata</code>는 <code>ndata</code>와 동일합니다.  
    튜토리얼 섹션 내의 모든 그래프는 동질적입니다.
</div>

예를 들어, 여기에서 GraphSAGE 컨볼루션을 DGL로 어떻게 구현하는지 보여줍니다.
For example, here is how you can implement GraphSAGE convolution in DGL.

In [2]:
import dgl.function as fn

class SAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model.
    
    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """
    def __init__(self, in_feat, out_feat):
        super(SAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)
    
    def forward(self, g, h):
        """Forward computation
        
        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        """
        with g.local_scope():
            g.srcdata['h'] = h
            # update_all is a message passing API.
            g.update_all(fn.copy_u('h', 'm'), fn.mean('m', 'h_neigh'))
            h_neigh = g.dstdata['h_neigh']
            h_total = torch.cat([h_dst, h_neigh], dim=1)
            return F.relu(self.linear(h_total))

코드의 가운데 부분은 `g.update_all` 함수인데, 이는 이웃 피처를 수집하고 평균을 내는 역할을 합니다.  

여기에 총 3개의 개념이 등장합니다.  

* Message 함수 `fn.copy_u('h', 'm')`는 *messages*가 이웃에 전달될 때 '`h`'의 노드 피처를 복사함
* Reduce 함수 `fn.mean('m', 'h_neigh')`는 모든 수신된 `'m'`의 message를 평균내고 그 결과를 새로운 노드 피처 `'h_neigh'`에 저장함.
* `update_all`은 DGL에게 message를 시작하고 모든 노드와 엣지에 대해 reduce 함수를 실행하게 합니다.



## 더욱 정밀한 커스터마이징

DGL에서는, `dgl.function` 패키지에서 많은 built-in message와 reduce 함수를 제공합니다. 

![api](../asset/dgl-mp.png)

더 많은 정보는 [the API doc](https://docs.dgl.ai/api/python/function.html)에서 보실 수 있습니다.

이 API들은 새로운 그래프 컨볼루션 모듈을 빠르게 구현할 수 있도록 해줍니다.  
예를 들어, 아래는  이웃의 표현을 가중 평균으로 통합하는 새로운 `SAGEConv`를 구현합니다.  
`edata`가 message passing에 참여할 수도 있는 엣지 피처를 가지고 있을 수 있다는 것을 주목해 주세요.

In [3]:
class SAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model.
    
    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """
    def __init__(self, in_feat, out_feat):
        super(SAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)
    
    def forward(self, g, h, w):
        """Forward computation
        
        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        w : Tensor
            The edge weight.
        """
        h_dst = h[:g.number_of_dst_nodes()]
        with g.local_scope():
            g.srcdata['h'] = h
            g.edata['w'] = w
            # update_all is a message passing API.
            g.update_all(fn.u_mul_e('h', 'w', 'm'), fn.mean('m', 'h_neigh'))
            h_neigh = g.dstdata['h_neigh']
            h_total = torch.cat([h_dst, h_neigh], dim=1)
            return F.relu(self.linear(h_total))

## 사용자 정의 함수를 통한 훨씬 더 정교한 커스터마이징

DGL은 최고의 자유도를 위해 사용자 정의 message와 reduce 함수를 허용합니다.  
여기에서, 사용자 정의 message 함수는 `fn.u_mul_e('h', 'w', 'm')`와 동일합니다.

In [4]:
def u_mul_e_udf(edges):
    return {'m' : edges.src['h'] * edges.data['w']}

`edges`는 3개로 구성되어 있습니다. `src`, `data` 그리고 `dst`입니다.  
소스 노드 피처, 엣지 피처, 목적지 노드 피처를 모든 엣지에 대해 표현해 줍니다.

## Recap
## 복습

* `srcdata` 와 `dstdata`를 인풋 노드 피처를 할당하고 아웃풋 노드 피처를 가져오는 데 사용하세요.
* `dgl.function`의 built-in message와 reduce 함수를 사용해 새로운 NN 모듈을 커스터마이징 하세요.
* 사용자 정의 함수는 훨씬 더 정교한 커스터마이징을 제공합니다.